# Data cleaning for test.csv set

In order to use the trained model on the test.csv dataset, we have to ensure the test dataframe matches the train dataframe. 

In this section, we will:
1. Check for duplicates
2. Drop columns that were dropped in the training set
3. Check for null values and drop these values, if any
4. Binarize `commercial`, `market_hawker`, `multistorey_carpark`, `precinct_pavilion` columns like in the training dataset
5. Add `age_at_tranc` and `resale_price` columns like in the training dataset
6. Rearrange columns so that `resale_price` is the first column in the dataset
7. Check columns and dtypes match those in training set
8. Pickle the test dataset to use in the modelling code notebook (02_model preparation and fitting)

In [1]:
# import libraries

import pandas as pd
import numpy as np

In [2]:
# open test.csv file

hdb_test_og = pd.read_csv("data/test.csv")

/var/folders/5q/pll34czs1rg04zxgqg6sl2qw0000gn/T/ipykernel_53316/1843630832.py:3: DtypeWarning: Columns (40) have mixed types. Specify dtype option on import or set low_memory=False.
  hdb_test_og = pd.read_csv("data/test.csv")


In [3]:
hdb_test_og.shape

(16737, 76)

In [4]:
# check duplicates

hdb_test_og.duplicated().value_counts()

False    16737
dtype: int64

In [5]:
# check null values

hdb_test_og.isnull().sum()

id                   0
Tranc_YearMonth      0
town                 0
flat_type            0
block                0
                    ..
sec_sch_name         0
cutoff_point         0
affiliation          0
sec_sch_latitude     0
sec_sch_longitude    0
Length: 76, dtype: int64

1. Impute missing values into `Mall_Nearest_Distance` by using the average of the other values
2. Other columns with null values will be dropped later as not used in test dataframe

In [6]:
# impute average into null values for Mall_Nearest_Distance column

average_distance = hdb_test_og['Mall_Nearest_Distance'].mean()

from sklearn.impute import SimpleImputer

imp_mean = SimpleImputer(missing_values=np.NaN, strategy='constant', fill_value=average_distance)
hdb_test_og['Mall_Nearest_Distance'] = imp_mean.fit_transform(hdb_test_og['Mall_Nearest_Distance'].values.reshape(-1,1))

In [7]:
# drop columns that were dropped in training set

to_drop = ['id', 'Tranc_YearMonth', 'block', 'street_name', 'storey_range', 'floor_area_sqm', 'lower', 'upper', 'mid', 'full_flat_type', 'address', 'hdb_age', 'year_completed', 'residential',
           'postal', 'Latitude', 'Longitude', 'Mall_Within_500m', 'Mall_Within_1km', 'Mall_Within_2km', 'Hawker_Within_500m', 'Hawker_Within_1km', 'Hawker_Within_2km', 
           'mrt_latitude', 'mrt_longitude','bus_stop_name', 'bus_stop_latitude', 'bus_stop_longitude', 'pri_sch_latitude', 'pri_sch_longitude', 'sec_sch_latitude', 'sec_sch_longitude']

hdb_test = hdb_test_og.drop(columns = to_drop)

In [8]:
to_binarize = ['commercial', 'market_hawker', 'multistorey_carpark', 'precinct_pavilion']

for i in range(0,len(to_binarize)):
    hdb_test[to_binarize[i]] = hdb_test[to_binarize[i]].map({'Y': 1, 'N': 0})

In [9]:
# hdb age at year of transaction

hdb_test['age_at_tranc'] = hdb_test['Tranc_Year'] - hdb_test['lease_commence_date']

In [10]:
hdb_test['resale_price'] = None

In [11]:
hdb_test.insert(0, 'resale_price', hdb_test.pop('resale_price'))

In [12]:
hdb_test['resale_price'] = hdb_test['resale_price'].astype(float)

In [13]:
# open hdb test set

hdb = pd.read_pickle("data/hdb_clean.pkl")

In [14]:
# write function to check for matching

def check_match(list1, list2):
    if len(list1) != len(list2):
        print('The shapes do not match')
    else:
        for i in range(0, len(list1)):
            if list1[i] != list2[i]:
                print(f'Difference in index {i}')
                break
        else:
            print('Match')

In [15]:
# check that hdb and hdb_test columns match

check_match(hdb.columns, hdb_test.columns)

Match


In [16]:
# check that hdb and hdb_test dtypes match

check_match(hdb.dtypes, hdb_test.dtypes)

Match


In [17]:
# pickle original and cleaned dataset

hdb_test_og.to_pickle('./data/hdb_test_original.pkl')
hdb_test.to_pickle('./data/hdb_test_clean.pkl')